DAY 1: POINTS

In [ ]:
# Import necessary libraries
import folium
import geopandas as gpd
import matplotlib.pyplot as plt
from folium.plugins import MarkerCluster, HeatMap

# read hospital data
health = gpd.read_file("hospital_TZ.geojson")

# convert to WGS84
points = health.to_crs("epsg:4326")

# read Tanzania boundary data
tza = gpd.read_file(r"boundary/tza_admbnda_adm2_20181019.shp")

# convert to WGS84
tza = tza.to_crs("epsg:4326")

# Create a base map centered on Tanzania
base = folium.Map(location=[-6.3690, 34.8888], zoom_start=6, tiles="CartoDB positron")

# Add Tanzania boundaries to the map
folium.GeoJson(tza,name="Districts",style_function=lambda x:{"fillcolor":"none",
                                           "color":"black", "weight":1.5}, 
               tooltip=folium.GeoJsonTooltip(fields=["ADM2_EN"],aliases=["District:"])).add_to(base)  


# Add hospital points with clustering
marker_cluster = MarkerCluster(name="Hospitals").add_to(base)
for _, row in health.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=5,
        color="d73027",
        fill=True,
        fill_color="d73027",
        fill_opacity=0.6,
        popup=row.get("NAME","Hospital")
    ).add_to(marker_cluster)
    
# Add heatmap layer to visualize hospital density
hospital_coords = [[row.geometry.y, row.geometry.x] for _, row in health.iterrows()]
HeatMap(
    hospital_coords,
    radius=15,
    blur=12,
    gradient={0.2:"yellow",0.04:"orange",0.06:"red",0.08:
        "darkred"},name="Hospital Density").add_to(base)

# Add layer control to toggle layers
folium.LayerControl(collapsed=False).add_to(base)

# Save the map to an HTML file
base.save("tanzania_hospital_map_challenge.html")